In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import sys
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Importing keras from tensorflow
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, GRU, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay

sys.path.append('../')
from label_algorithms import oracle
import LSTM as lstm_impl
import CNN_LSTM as cnn_lstm_impl
import utils as model_utils
import transformer as tr_impl

In [3]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is NOT available")


GPU is NOT available


In [4]:
ticker_symbol = 'GC=F'
start_date = '2000-01-01'
end_date = '2023-11-01'

prices = yf.download(ticker_symbol, start_date, end_date, interval='1d')
prices.index = prices.index.tz_localize(None)

print(prices.shape)

[*********************100%%**********************]  1 of 1 completed
(5813, 6)


In [5]:
fee = 0.0004
labels = oracle.binary_trend_labels(prices['Close'], fee=fee)
print(labels.shape)

(5813,)


In [6]:
window_size = 40
X = model_utils.get_X(prices, window_size)[:-1]
Y = model_utils.get_Y(labels, window_size)
print(X.shape, Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)
print(f'Train X: {X_train.shape}, Y: {Y_train.shape}')
print(f'Test X: {X_test.shape}, Y: {Y_test.shape}')

batch_size = int(X_train.shape[0]/10)

(5772, 40, 6) (5772, 1)
Train X: (4617, 40, 6), Y: (4617, 1)
Test X: (1155, 40, 6), Y: (1155, 1)


## LSTM

In [8]:
# Create a learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# Create an Adam optimizer with the learning rate schedule
opt = Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False)

# Build the LSTM model
lstm_model = lstm_impl.model(X.shape[1], X.shape[2])

# Compile the model
lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

lstm_model.fit(X_train,Y_train,epochs=2,batch_size=batch_size,verbose=1)

lstm_model.save('./saved_models/lstm_model.keras')

Epoch 1/2
11/11 [==============================] - 7s 274ms/step - loss: 0.6551 - accuracy: 0.6424
Epoch 2/2
11/11 [==============================] - 3s 251ms/step - loss: 0.6370 - accuracy: 0.6688


In [9]:
loaded_model = load_model('./saved_models/lstm_model.keras')
loaded_model.evaluate(X_test,Y_test)

37/37 [==============================] - 2s 19ms/step - loss: 0.6424 - accuracy: 0.6589


[0.6423724293708801, 0.6588744521141052]

## CNN-LSTM

In [16]:
# Create a learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# Create an Adam optimizer with the learning rate schedule
opt = Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False)

n_steps = 4
n_length = 10
n_fetures = X_train.shape[2]
X_train_r = X_train.reshape((X_train.shape[0], n_steps, n_length, n_fetures))
X_test_r = X_test.reshape((X_test.shape[0], n_steps, n_length, n_fetures))
print(X_train_r.shape, X_test_r.shape)

# Build the CNN-LSTM model
cnn_lstm_model = cnn_lstm_impl.model(n_length, n_fetures)

# Compile the model
cnn_lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

cnn_lstm_model.fit(X_train_r,Y_train,epochs=1000,batch_size=batch_size,verbose=1)

(5280, 4, 10, 6) (1321, 4, 10, 6)
Epoch 1/1000
10/10 [==============================] - 3s 46ms/step - loss: 0.6962 - accuracy: 0.5267
Epoch 2/1000
10/10 [==============================] - 0s 46ms/step - loss: 0.6874 - accuracy: 0.5521
Epoch 3/1000
10/10 [==============================] - 0s 46ms/step - loss: 0.6869 - accuracy: 0.5521
Epoch 4/1000
10/10 [==============================] - 0s 46ms/step - loss: 0.6860 - accuracy: 0.5521
Epoch 5/1000
10/10 [==============================] - 0s 49ms/step - loss: 0.6831 - accuracy: 0.5521
Epoch 6/1000
10/10 [==============================] - 0s 46ms/step - loss: 0.6806 - accuracy: 0.5521
Epoch 7/1000
10/10 [==============================] - 0s 46ms/step - loss: 0.6807 - accuracy: 0.5754
Epoch 8/1000
10/10 [==============================] - 0s 49ms/step - loss: 0.6779 - accuracy: 0.5794
Epoch 9/1000
10/10 [==============================] - 0s 45ms/step - loss: 0.6809 - accuracy: 0.5871
Epoch 10/1000
10/10 [==============================] - 0s

In [17]:
cnn_lstm_model.evaluate(X_test_r,Y_test)

42/42 [==============================] - 1s 4ms/step - loss: 0.7123 - accuracy: 0.5223


[0.7123053073883057, 0.5223315954208374]

## Transformer

In [16]:
input_shape = X_train.shape[1:]

tr_model = tr_impl.build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

tr_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy'],
)
tr_model.summary()

tr_model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=batch_size,
)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 40, 6)]              0         []                            
                                                                                                  
 layer_normalization_24 (La  (None, 40, 6)                12        ['input_4[0][0]']             
 yerNormalization)                                                                                
                                                                                                  
 multi_head_attention_12 (M  (None, 40, 6)                27654     ['layer_normalization_24[0][0]
 ultiHeadAttention)                                                 ',                            
                                                                     'layer_normalization_24

In [14]:
tr_model.evaluate(X_test, Y_test, verbose=1)

42/42 [==============================] - 4s 94ms/step - loss: 0.7153 - accuracy: 0.5223


[0.7153221964836121, 0.5223315954208374]